In [34]:
# Ram Ram

In [35]:
import networkx as nx
import random

In [36]:
commodities=['P1','P2','P3','P4']
total_nodes=['A','B','C','D','E','F','G']
depot_nodes=['A','B']
customer_nodes=['C','D','E','F','G']

vehicle_content=0
with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\vessel.txt") as f:
    vehicle_content=f.read()
vehicle_content=vehicle_content.split("\n")
if vehicle_content[-1] =='' or vehicle_content[-1] =='\n':
    vehicle_content=vehicle_content[:-1]
vehicle_dict={}
for item in vehicle_content:
    item=item.split(":")
    vehicle_cap=int(item[1].split(",")[0].split("(")[1].strip())
    vehicle_cost=int(item[1].split(",")[1].split(")")[0].strip())
    vehicle_dict[item[0].strip()]=[vehicle_cap,vehicle_cost]

capacity_content=0
with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\capacity.txt") as f:
    capacity_content=f.read()
capacity_content=capacity_content.split("\n")

if capacity_content[-1]==" " or capacity_content[-1]=="\n":
    capacity_content=capacity_content[:-1]

capacity_dict={}
for item in capacity_content:
    item=item.split(":")
    depot_id=item[0].split(",")[0].split("(")[1].strip()
    product_id=item[0].split(",")[1].split(")")[0].strip()
    depot_cap=int(item[1].strip())
    capacity_dict[(depot_id,product_id)]=depot_cap

time_content=0
with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\time.txt") as f:
    time_content=f.read()
time_content=time_content.split("\n")

if time_content[-1]==" " or time_content[-1]=="\n":
    time_content=time_content[:-1]
mapper={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G'}
time_dict={}
for item in time_content:
    item=item.split(":")
    source_depot=int(item[0].split(",")[0].split("(")[1].strip())
    destination_depot=int(item[0].split(",")[1].split(")")[0].strip())
    time_to_travel=int(item[1].strip())
    time_dict[(mapper[source_depot],mapper[destination_depot])]=time_to_travel
    time_dict[(mapper[destination_depot],mapper[source_depot])]=time_to_travel

demand_content=0
with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\demand.txt") as f:
    demand_content=f.read()
demand_content=demand_content.split("\n")

if demand_content[-1]==" " or demand_content[-1]=="\n":
    demand_content=demand_content[:-1]

# demand_dict={}
# for item in demand_content:
#     item=item.split(":")
#     depot_id=item[0].split(",")[0].split("(")[1].strip()
#     product_id=item[0].split(",")[1].split(")")[0].strip()
#     demand_for_product=int(item[1].strip())
#     demand_dict[(depot_id,product_id)]=demand_for_product
# for node in depot_nodes:
#     for commodity in commodities:
#         demand_dict[(node,commodity)]=0


In [37]:
days=[]
for i in range(1,16):
    days.append('D'+str(i))

demand_low_param=10000
demand_up_param=15000

demand_dict={}

for node in total_nodes:
    for c in range(len(commodities)):
        for d in range(len(days)):
            if node in depot_nodes:
                demand_dict[(node,commodities[c],days[d])]=0
            else:
                demand_dict[(node,commodities[c],days[d])]=random.randint(demand_low_param,demand_up_param)



In [38]:
G=nx.Graph()
for node in total_nodes:
    G.add_node(node)
for edge in time_dict.keys():
    if edge[0]!=edge[1]:
        G.add_edge(edge[0],edge[1],weight=time_dict[edge])


In [39]:
from itertools import combinations,permutations

global_paths_set=set()
all_paths_list=[]
roundtrips_dict={}

for depot in depot_nodes:
    roundtrips_dict[depot]=set()
    for node in total_nodes:
        for path in list(nx.all_simple_paths(G,depot,node)):
            if depot not in path[1:]:
                temp=path.copy()
                temp.append(depot)
                temp=tuple(temp)
                roundtrips_dict[depot].add((temp))

# for permutation in permutations(depot_nodes, 2):
#     source_depot=permutation[0]
#     destination_depot=permutation[1]
#     for path in list(nx.all_simple_paths(G,source=source_depot,target=destination_depot)):
#         temp=tuple(path)
#         all_paths_list.append(path)
#         global_paths_set.add(temp)

for depot in depot_nodes:
    for path in roundtrips_dict[depot]:
        global_paths_set.add(path)



        
        

In [40]:
# global_paths_set_copy=global_paths_set.copy()

# !!!!!!!!!!!!!!!!!!!! *************** WARNING WE HAVE CHANGED THE GLOBAL PATHS SET INTO A LIST FOR CONVENIENCE PURPOSES
# AS PYTHON WONT ALLOW SUBSCRIPTING A SET VIA POSITION INDEX

global_paths_set=list(global_paths_set)

path_dict={}
for keys,values in vehicle_dict.items():
    for c in range(len(commodities)):
        for d in range(len(days)):
            path_dict[(keys,commodities[c],days[d])]={}
            for path_index in range(len(global_paths_set)):
                temp={'CustomerSet':set(),'Start':-1,'End':-1,'DepotSet':set(),'Path':[],'TotalCustomerDemand':0,'VehicleCapacity':0,'EachCustomerDemand':[],'Day':days[d],'Commodity':commodities[c],'VehicleType':keys}

                #Setting start and end
                temp['Start']=global_paths_set[path_index][0]
                temp['End']=global_paths_set[path_index][-1]

                temp['Path']=global_paths_set[path_index]

                #Customer Sets and Depot Sets
                temp_customer_demand=0
                for i in range(len(global_paths_set[path_index])):
                    if global_paths_set[path_index][i] in customer_nodes:
                        temp['CustomerSet'].add(global_paths_set[path_index][i])
                        temp_customer_demand+=(demand_dict[(global_paths_set[path_index][i],commodities[c],days[d])])
                    elif global_paths_set[path_index][i] in depot_nodes:
                        temp['DepotSet'].add(global_paths_set[path_index][i])
                    temp['EachCustomerDemand'].append(demand_dict[(global_paths_set[path_index][i],commodities[c],days[d])])
                
                temp['TotalCustomerDemand']=temp_customer_demand
                temp['VehicleCapacity']=values[0]
                if temp['VehicleCapacity'] >= temp['TotalCustomerDemand'] and capacity_dict[(temp['Start'],commodities[c])] >= temp['TotalCustomerDemand']:
                    temp['Feasible']= True
                else:
                    temp['Feasible']=False
                
                temp['Cost']=0
                for i in range(len(global_paths_set[path_index])-1):
                    temp['Cost']+=time_dict[(global_paths_set[path_index][i],global_paths_set[path_index][i+1])]*vehicle_dict[keys][1]
                
                temp['ConsumptionTracker']=0

                
                temp['CustomerSet']=list(temp['CustomerSet'])
                temp['DepotSet']=list(temp['DepotSet'])
                if temp['Feasible']==True:
                    temp['ConsumptionTracker']=temp['TotalCustomerDemand']
                    path_dict[(keys,commodities[c],days[d])][path_index]=temp


feasible_path_dict=path_dict

In [42]:
feasible_vehicle_dict={}
for cust_node in customer_nodes:
    for c in range(len(commodities)):
        for d in range(len(days)):
            feasible_vehicle_dict[(cust_node,commodities[c],days[d])]=[]
            for vehicle_index,v1 in vehicle_dict.items():
                if demand_dict[(cust_node,commodities[c],days[d])]<=v1[0]:
                    feasible_vehicle_dict[(cust_node,commodities[c],days[d])].append(vehicle_index)
    



In [43]:
total_demand_to_be_satisfied=0
for cust_node in customer_nodes:
    for c in range(len(commodities)):
        for d in range(len(days)):
            total_demand_to_be_satisfied+=demand_dict[(cust_node,commodities[c],days[d])]


In [44]:


z_decision_variable_indices=[]
objective_function_values={}
for vehicle_index,v1 in vehicle_dict.items():
    for c in range(len(commodities)): 
        for d in range(len(days)):
            for path_index,v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                    z_decision_variable_indices.append((vehicle_index,path_index,commodities[c],days[d]))
                    objective_function_values[(vehicle_index,path_index,commodities[c],days[d])]= feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Cost']-( (10**10) *feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['ConsumptionTracker']) # + f
                    

from docplex.mp.model import Model

vrp_model = Model('Heterogenous Fleet MDMT-VRP')
z=vrp_model.binary_var_dict(z_decision_variable_indices,name='z')


obj_fn=0
for vehicle_index,v1 in vehicle_dict.items():
    for c in range(len(commodities)):
        for d in range(len(days)):
            for path_index,v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                obj_fn+=objective_function_values[(vehicle_index,path_index,commodities[c],days[d])]*z[(vehicle_index,path_index,commodities[c],days[d])]

obj_fn+=total_demand_to_be_satisfied * (10**10)


vrp_model.set_objective("min",obj_fn)

for j in customer_nodes:
    for c in range(len(commodities)):
        for d in range(len(days)):
            c_once=0
            for vehicle_index,v1 in vehicle_dict.items():
                if vehicle_index in feasible_vehicle_dict[(j,commodities[c],days[d])]:         
                    for path_index,v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                        if j in feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['CustomerSet']:
                            c_once+=z[(vehicle_index,path_index,commodities[c],days[d])]
                else:
                    continue
            if c_once:
                #print("BRO")
                vrp_model.add_constraint(c_once<=1,'cvo') #customer visit once(cvo)



for i in depot_nodes:
    for c in range(len(commodities)):
        depot_cap_constraint=0
        for vehicle_index,v1 in vehicle_dict.items():
            for d in range(len(days)):
                for path_index,v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                    if feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Start'] == i:
                        depot_cap_constraint+=(feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['ConsumptionTracker']*z[(vehicle_index,path_index,commodities[c],days[d])]) 
        if depot_cap_constraint:
            #print("HELLO",i,commodities[c])
            vrp_model.add_constraint(depot_cap_constraint <= capacity_dict[(i,commodities[c])], 'dcc') #depot capacity constraint(dcc)

# #------- --------------------------------!!!!!!!!!!!!!!!!!!!!!!!!!!

for i in depot_nodes:
    for vehicle_index, v1 in vehicle_dict.items():
        for c in range(len(commodities)):
            for d in range(len(days)):
                lhs=0
                for j in depot_nodes:
                    for path_index, v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                        if feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Start']==i and feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['End']==j:
                            lhs+=z[(vehicle_index,path_index,commodities[c],days[d])]
                
                rhs=0
                for j in depot_nodes:
                    for path_index, v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                        if feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Start']==j and feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['End']==i:
                            rhs+=z[(vehicle_index,path_index,commodities[c],days[d])]

                #print("hhh",i,vehicle_index,commodities)
                vrp_model.add_constraint(lhs==rhs, 'dbc') #depot balance constraint(dbc)
            


sol=vrp_model.solve()


    

sol_values=sol.get_value_dict(z)
for k,v in sol_values.items():
    if v==1:
        print(True,k)


sol.get_objective_value()




True ('V4', 2773, 'P1', 'D1')
True ('V4', 3381, 'P1', 'D1')
True ('V4', 170, 'P1', 'D4')
True ('V4', 1801, 'P1', 'D7')
True ('V4', 3392, 'P1', 'D8')
True ('V4', 330, 'P1', 'D9')
True ('V4', 1832, 'P1', 'D10')
True ('V4', 1366, 'P1', 'D11')
True ('V4', 1366, 'P1', 'D13')
True ('V4', 1897, 'P1', 'D15')
True ('V4', 3787, 'P2', 'D3')
True ('V4', 1366, 'P2', 'D6')
True ('V4', 2574, 'P2', 'D7')
True ('V4', 3381, 'P2', 'D8')
True ('V4', 1366, 'P2', 'D10')
True ('V4', 2574, 'P3', 'D1')
True ('V4', 2574, 'P3', 'D2')
True ('V4', 2574, 'P3', 'D3')
True ('V4', 2574, 'P3', 'D4')
True ('V4', 2574, 'P3', 'D5')
True ('V4', 2574, 'P3', 'D6')
True ('V4', 2574, 'P3', 'D7')
True ('V4', 1366, 'P3', 'D8')
True ('V4', 2574, 'P3', 'D9')
True ('V4', 1366, 'P3', 'D10')
True ('V4', 2574, 'P3', 'D11')
True ('V4', 1366, 'P3', 'D12')
True ('V4', 1366, 'P3', 'D13')
True ('V4', 1897, 'P3', 'D14')
True ('V4', 2574, 'P3', 'D15')
True ('V4', 1366, 'P4', 'D3')
True ('V4', 2531, 'P4', 'D9')
True ('V4', 2322, 'P4', 'D10')


1.951380000121545e+16

In [45]:
for k,v in sol_values.items():
    if v==1:
        print(True,k)


True ('V4', 2773, 'P1', 'D1')
True ('V4', 3381, 'P1', 'D1')
True ('V4', 170, 'P1', 'D4')
True ('V4', 1801, 'P1', 'D7')
True ('V4', 3392, 'P1', 'D8')
True ('V4', 330, 'P1', 'D9')
True ('V4', 1832, 'P1', 'D10')
True ('V4', 1366, 'P1', 'D11')
True ('V4', 1366, 'P1', 'D13')
True ('V4', 1897, 'P1', 'D15')
True ('V4', 3787, 'P2', 'D3')
True ('V4', 1366, 'P2', 'D6')
True ('V4', 2574, 'P2', 'D7')
True ('V4', 3381, 'P2', 'D8')
True ('V4', 1366, 'P2', 'D10')
True ('V4', 2574, 'P3', 'D1')
True ('V4', 2574, 'P3', 'D2')
True ('V4', 2574, 'P3', 'D3')
True ('V4', 2574, 'P3', 'D4')
True ('V4', 2574, 'P3', 'D5')
True ('V4', 2574, 'P3', 'D6')
True ('V4', 2574, 'P3', 'D7')
True ('V4', 1366, 'P3', 'D8')
True ('V4', 2574, 'P3', 'D9')
True ('V4', 1366, 'P3', 'D10')
True ('V4', 2574, 'P3', 'D11')
True ('V4', 1366, 'P3', 'D12')
True ('V4', 1366, 'P3', 'D13')
True ('V4', 1897, 'P3', 'D14')
True ('V4', 2574, 'P3', 'D15')
True ('V4', 1366, 'P4', 'D3')
True ('V4', 2531, 'P4', 'D9')
True ('V4', 2322, 'P4', 'D10')


In [46]:
# print(obj_fn)

In [55]:
sol_values=sol.get_value_dict(z)
sol_paths=[]
print("-----> FINAL OBJECTIVE VALUE : ", sol.get_objective_value())
print("\n\n\n")
demand_satisfied_by_model=0
for k,v in sol_values.items():
    if v==1:
        print("**********************************************************")
        print("VESSEL TYPE   : ",k[0])
        print("ROUTE NO      : ",k[1])
        print("PRODUCT TYPE  : ",k[2])
        print("START REFINERY: ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['Start'])
        print("END REFINERY  : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['End'])
        print("ROUTE         : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['Path'])
        print("ROUTECOST     : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['Cost'])
        print("DAY           : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['Day'])
        print("CONSUMPTION   : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['ConsumptionTracker'])
        demand_satisfied_by_model+=feasible_path_dict[(k[0],k[2],k[3])][k[1]]['ConsumptionTracker']
        sol_paths.append((k,feasible_path_dict[(k[0],k[2],k[3])][k[1]]))
        print("**********************************************************\n\n")


            

-----> FINAL OBJECTIVE VALUE :  1.951380000121545e+16




**********************************************************
VESSEL TYPE   :  V4
ROUTE NO      :  2773
PRODUCT TYPE  :  P1
START REFINERY:  B
END REFINERY  :  B
ROUTE         :  ('B', 'D', 'C', 'B')
ROUTECOST     :  35550
DAY           :  D1
CONSUMPTION   :  29072
**********************************************************


**********************************************************
VESSEL TYPE   :  V4
ROUTE NO      :  3381
PRODUCT TYPE  :  P1
START REFINERY:  A
END REFINERY  :  A
ROUTE         :  ('A', 'F', 'A')
ROUTECOST     :  13500
DAY           :  D1
CONSUMPTION   :  10842
**********************************************************


**********************************************************
VESSEL TYPE   :  V4
ROUTE NO      :  170
PRODUCT TYPE  :  P1
START REFINERY:  B
END REFINERY  :  B
ROUTE         :  ('B', 'G', 'E', 'D', 'C', 'B')
ROUTECOST     :  42300
DAY           :  D4
CONSUMPTION   :  50497
***************************

**********************************************************
VESSEL TYPE   :  V4
ROUTE NO      :  3689
PRODUCT TYPE  :  P4
START REFINERY:  B
END REFINERY  :  B
ROUTE         :  ('B', 'G', 'E', 'D', 'C', 'F', 'A', 'B')
ROUTECOST     :  45900
DAY           :  D14
CONSUMPTION   :  59937
**********************************************************


**********************************************************
VESSEL TYPE   :  V4
ROUTE NO      :  1366
PRODUCT TYPE  :  P4
START REFINERY:  A
END REFINERY  :  A
ROUTE         :  ('A', 'C', 'D', 'E', 'G', 'F', 'A')
ROUTECOST     :  36450
DAY           :  D15
CONSUMPTION   :  63566
**********************************************************




In [56]:
print("Total Demand to be satisfied = ",total_demand_to_be_satisfied)
print("Demand satisfied by model    = ",demand_satisfied_by_model)

Total Demand to be satisfied =  3771079
Demand satisfied by model    =  1819699


In [49]:
# overall_consumption={}
# for c in range(len(commodities)):
#     overall_consumption[commodities[c]]=0
# for item in sol_paths:
#     overall_consumption[item[0][2]]+=sum(list(item[1]['ConsumptionTracker'].values()))

# overall_demand={}
# for c in range(len(commodities)):
#     overall_demand[commodities[c]]=0
# for k,v in demand_dict.items():
#     overall_demand[k[1]]+=v

# for c in range(len(commodities)):
#     print("Demand for Product ",commodities[c]," = ",overall_demand[commodities[c]],"and Actual Vessel Consumption = ",overall_consumption[commodities[c]])